In [1]:
import numpy as np
import pandas as pd
from selenium import webdriver
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import time

In [ ]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.maximize_window()
kw_list = ['B2B','Wholesale','B2B Order','Wholesale order','Request For Quote','Lock','Hide price','Password',
           'Referral','Loyalty','Rewards','Label','Badge','Product Option',
          'Customize Product','Session recording','Visitor Replay','Store Locator','Locator Map','Subscriptions']
kw_list_1 = ['Wholesale','Wholesale order','Lock','Password','Loyalty','Label','Product Option','Session recording','Store Locator','Subscriptions']

element_list = []
ranking = 1
break_loop = False
condition_rank = 50
for i in kw_list_1:
    for j in range(1,4):
        if break_loop == True:
            ranking = 1
            break_loop = False
            break
        driver.get(f'https://apps.shopify.com/search?q={i}&page={j}')
        time.sleep(2)
        for h in range(1,25):
            if ranking > condition_rank:
                if j == 3:
                    break_loop = False
                    ranking = 1
                else:
                    break_loop = True
                break
            name = driver.find_element(By.XPATH,f'//*[@id="search_app_grid"]/div/div/div[{h}]/div/div/div/div[1]').text
            short_description = driver.find_element(By.XPATH,f'//*[@id="search_app_grid"]/div/div/div[{h}]/div/div/div/div[3]').text
            review = driver.find_element(By.XPATH,f'//*[@id="search_app_grid"]/div/div/div[{h}]/div/div/div/div[2]/span[3]').text
            rating = driver.find_element(By.XPATH,f'//*[@id="search_app_grid"]/div/div/div[{h}]/div/div/div/div[2]/span[1]').text[0:3]
            link = driver.find_element(By.XPATH,f'//*[@id="search_app_grid"]/div/div/div[{h}]/div/div/div/div[1]/a').get_attribute('href')
            driver.get(link)
            time.sleep(2)
            owner = driver.find_element(By.XPATH,'//*[@id="adp-hero"]/div/div/div[1]/div/div[1]/div[2]/div[2]/div[3]/div').text
            full_description =  driver.find_element(By.XPATH,'//*[@id="adp-details-section"]/div/div/div[2]').text
            try:
                pricing_plan = driver.find_element(By.XPATH,'//*[@id="adp-pricing"]/div/div/div/div[2]').text
            except NoSuchElementException:
                print('No plan')
                pricing_plan = 'No plan'
            element_item = {
                    'Name':name,
                    'Link':link,
                    'Owner':owner,
                    'Short_description': short_description,
                    'Rating':rating,
                    'Review': review,
                    'Full_description': full_description,
                    'Pricing_plan': pricing_plan,
                    'ranking':ranking
                         }
            element_list.append(element_item)
            ranking = ranking + 1 
            driver.execute_script("window.history.go(-1)")
            time.sleep(2)
                
    print(f"Find key word {i}")
                    
    

C:\Users\BSS\AppData\Local\Temp\ipykernel_19520\4067492290.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [ ]:
df_1 = pd.DataFrame(element_list)
df_1

In [ ]:
#clean data
# link_column:
df_1['Link'] = df_1['Link'].apply(lambda x: pd.Series(str(x).split("?")))[0] 
df_1['Link']
#Review_column:
review_list = []
for i in df_1.Review:
    rw = i[1:-1]
    review_list.append(rw)
df_1.Review = review_list

In [ ]:
#Import data into file data ranking
import gspread
from gspread_dataframe import set_with_dataframe
from google.oauth2.service_account import Credentials
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
import pygsheets

client = pygsheets.authorize(service_account_file="b2breviews-018c032a2741.json")
sh=client.open('Linh - Full Shopify App Market Research')
wks = sh.worksheet_by_title('Ranking Data') # open the existing file
cells = wks.get_all_values(include_tailing_empty_rows=False, include_tailing_empty=False, returnas='matrix')
last_row_1 = len(cells)+1
wks.set_dataframe(df_1,(last_row_1,0),copy_head = False,extend = True )